In [62]:
import MDAnalysis as mda
import rdkit.Chem.AllChem
from openff.toolkit import Molecule, Topology
from openff.units import unit


def topology_from_pdb(
    pdbfile: str,
    vdwradii: dict[str, float] = {},
    overlap_factor: float = 0.55,
    shortest_bond: float = 0.1,
):
    """Use MDAnalysis' PDB loader to load a PDB file.

    Bonds not explicitly included in CONECT records are guessed from atom-atom
    distances. The `vdw_radii`, `required_overlap`, and `lower_bound` parameters
    configure this guessing behavior. The PDB file's CRYST1 records are used
    to compute atom-atom distances.

    Parameters
    ==========

    pdbfile
        Path to the PDB file to load.
    vdwradii
        Additional van der Waals radii used to guess bonds. A dictionary mapping
        elements to radii in Angstroms that augments the table provided at
        `MDAnalysis.topology.tables.vdwradii`.
    overlap_factor
        The maximum proportion of the sum of two atom's van der Waals radii at
        which distance the two atoms are considered bonded. Larger values will
        tend to produce more bonds.
    shortest_bond
        Any bonds found that are shorter than this distance in Angstroms are
        discarded.

    """
    u = mda.Universe(
        pdbfile,
        guess_bonds=True,
        vdwradii=vdwradii,
        fudge_factor=overlap_factor,
        lower_bound=shortest_bond,
    )

    rdmol = u.atoms.convert_to("RDKIT")
    rdmols = rdkit.Chem.AllChem.GetMolFrags(rdmol, asMols=True)

    offmols = [Molecule.from_rdkit(rdmol) for rdmol in rdmols]
    offtop = Topology.from_molecules(offmols)
    if u.dimensions is not None:
        offtop.box_vectors = (
            mda.lib.mdamath.triclinic_vectors(u.dimensions) * unit.angstrom
        )

    return offtop

In [52]:
offtop = topology_from_pdb("6anm_fixed.pdb.gz")

offtop.visualize()

NGLWidget()